## Text Summarization

In [1]:
from datasets import load_dataset, Dataset

# Stream and take only the first 20_000 samples from the Spanish and English datasets
spanish_stream = load_dataset("intfloat/multilingual_cc_news", "es", split="train", streaming=True).take(20000)
english_stream = load_dataset("intfloat/multilingual_cc_news", "en", split="train", streaming=True).take(20000)

# Convert streams to datasets
spanish_dataset = Dataset.from_list(list(spanish_stream))
english_dataset = Dataset.from_list(list(english_stream))

english_dataset

README.md: 0.00B [00:00, ?B/s]

multilingual_cc_news.py: 0.00B [00:00, ?B/s]

Dataset({
    features: ['title', 'maintext', 'url', 'date_publish'],
    num_rows: 20000
})

In [2]:
# Convert to DatasetDict
from datasets import DatasetDict

def split_dataset(dataset, train_size=15_000, val_size=2_500, test_size=2_500):
    train_dataset = dataset.train_test_split(
        train_size=train_size, 
        seed=42)
    val_dataset = train_dataset['test'].select(range(val_size))
    test_dataset = train_dataset['test'].select(range(val_size, val_size + test_size))
    return DatasetDict({
        'train': train_dataset['train'],
        'validation': val_dataset,
        'test': test_dataset
    })
    
spanish_dataset = split_dataset(spanish_dataset)
english_dataset = split_dataset(english_dataset)
english_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'maintext', 'url', 'date_publish'],
        num_rows: 15000
    })
    validation: Dataset({
        features: ['title', 'maintext', 'url', 'date_publish'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['title', 'maintext', 'url', 'date_publish'],
        num_rows: 2500
    })
})

In [7]:
# Show a few samples
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset['train'].shuffle(seed=seed).select(range(num_samples))
    for i, item in enumerate(sample):
        print(f"Title: {item['title']}")
        print(f"Maintext (first 200 words): {item['maintext'][:200]}\n")
show_samples(english_dataset)

Title: Victim of attempted abduction near RAF Marham to relive ordeal for Crimewatch
Maintext (first 200 words): A serviceman at the centre of an attempted abduction near RAF Marham is to relive his ordeal on tomorrow night’s BBC Crimewatch programme.
The victim, a married airman in his late 20s, has revealed he

Title: Saipan delegation OKs San Roque rezoning
Maintext (first 200 words): AFTER a lengthy discussion, the Saipan and Northern Islands Legislative Delegation on Tuesday passed a local measure amending the Saipan Zoning Law to rezone parts of San Roque.
Fifteen members voted 

Title: An unusual new late-night competitor for ESPN
Maintext (first 200 words): NEW YORK (AP) — To a certain segment of the population, Scott Van Pelt is a more popular late-night television star than Jimmy Fallon, Jimmy Kimmel and Stephen Colbert.
That segment — young men aged 1



In [8]:
# Concatenate the above two datasets (and rename it to be the same as in the LLM course)
from datasets import concatenate_datasets, DatasetDict
books_dataset = DatasetDict()

for split in english_dataset.keys():
    books_dataset[split] = concatenate_datasets(
        [english_dataset[split], spanish_dataset[split]])
    books_dataset[split] = books_dataset[split].shuffle(seed=42)
    
show_samples(books_dataset)

Title: Declararán el 21 de octubre como Día de Héctor Espino
Maintext (first 200 words): HERMOSILLO, Sonora(GH)
En la sesiÃ³n ordinaria correspondiente a septiembre, el Cabildo aprobÃ³ por unanimidad la propuesta de declarar el 21 de octubre como el DÃ­a de HÃ©ctor Espino, en honor al leg

Title: Tropical Storm Lisa forms over eastern tropical Atlantic
Maintext (first 200 words): 

Title: Earthquake felt from Nebraska to Texas
Maintext (first 200 words): PAWNEE, Okla. (AP) — One of Oklahoma’s largest earthquakes on record rattled other parts of the Midwest on Saturday from Nebraska to North Texas, and likely will turn new attention to the practice of 



In [9]:
# Filter out samples with empty title or maintext
def filter_empty_samples(dataset):
    return dataset.filter(lambda x: len(x['title'].strip()) > 0 and len(x['maintext'].strip()) > 0)
books_dataset = DatasetDict({
    split: filter_empty_samples(books_dataset[split]) for split in books_dataset.keys()
})
books_dataset

Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'maintext', 'url', 'date_publish'],
        num_rows: 28241
    })
    validation: Dataset({
        features: ['title', 'maintext', 'url', 'date_publish'],
        num_rows: 4701
    })
    test: Dataset({
        features: ['title', 'maintext', 'url', 'date_publish'],
        num_rows: 4699
    })
})

In [10]:
show_samples(books_dataset)

Title: Shimon Peres witnessed Israel’s history, and shaped it
Maintext (first 200 words): JERUSALEM (AP) — At every corner of Israel’s tumultuous history, Shimon Peres was there.
He was a young aide to the nation’s founding fathers when the country declared independence in 1948, and he pla

Title: Police: Boy, 2, dies after accidentally shooting self in chest
Maintext (first 200 words): QUAKERTOWN, Pa. — State police say a 2-year-old boy accidentally shot himself to death in a Pennsylvania home.
Authorities say the boy was pronounced dead Monday shortly after suffering a single gunsh

Title: Chiefs' Jamaal Charles doubtful for Week 2 at Houston
Maintext (first 200 words): Chiefs running back Jamaal Charles is doubtful for Sunday's game at Houston as he continues to work his way back from surgery last season to repair the torn ACL in his right knee.



In [13]:
# Load mt5-small model
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/workspace/projects/hf_llm/.venv/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [14]:
inputs = tokenizer("I love reading the Hunger Games!")
inputs

{'input_ids': [336, 3869, 11807, 287, 62893, 295, 12507, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁I', '▁love', '▁reading', '▁the', '▁Hung', 'er', '▁Games', '!', '</s>']

In [16]:
# Preprocessing the datasets
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["maintext"], max_length=max_input_length, truncation=True)
    labels = tokenizer(
        examples["title"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_books = books_dataset.map(
    preprocess_function, 
    batched=True,
)

Map:   0%|          | 0/28241 [00:00<?, ? examples/s]

Map:   0%|          | 0/4701 [00:00<?, ? examples/s]

Map:   0%|          | 0/4699 [00:00<?, ? examples/s]

In [17]:
tokenized_books

DatasetDict({
    train: Dataset({
        features: ['title', 'maintext', 'url', 'date_publish', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 28241
    })
    validation: Dataset({
        features: ['title', 'maintext', 'url', 'date_publish', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4701
    })
    test: Dataset({
        features: ['title', 'maintext', 'url', 'date_publish', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4699
    })
})

In [39]:
tokenized_datasets = books_dataset.map(
    preprocess_function, 
    batched=True,
)

Map:   0%|          | 0/28241 [00:00<?, ? examples/s]

Map:   0%|          | 0/4701 [00:00<?, ? examples/s]

Map:   0%|          | 0/4699 [00:00<?, ? examples/s]

In [18]:
# Rouge metric
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

import evaluate
rouge_score = evaluate.load("rouge")
results = rouge_score.compute(
    predictions=[generated_summary], 
    references=[reference_summary]
)
results

{'rouge1': np.float64(0.923076923076923),
 'rouge2': np.float64(0.7272727272727272),
 'rougeL': np.float64(0.923076923076923),
 'rougeLsum': np.float64(0.923076923076923)}

In [21]:
# Baseline case using NLTK
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [22]:
from nltk.tokenize import sent_tokenize
def three_sentence_summary(text):
    sentences = sent_tokenize(text)
    return ' '.join(sentences[:3])
three_sentence_summary(books_dataset['train'][0]['maintext'])

'NEW YORK (AP) — A week before the first presidential debate, Donald Trump is putting moderators on notice that he’ll be watching to see if they get too rough on him. In a series of interviews over the past week, the Republican nominee has asserted that “the system is being rigged” against him. The first of three scheduled debates between Trump and Hillary Clinton will be held on Sept. 26, with NBC’s Lester Holt as the journalist questioning the candidates.'

In [23]:
books_dataset['train'][0]['maintext']

'NEW YORK (AP) — A week before the first presidential debate, Donald Trump is putting moderators on notice that he’ll be watching to see if they get too rough on him.\nIn a series of interviews over the past week, the Republican nominee has asserted that “the system is being rigged” against him. The first of three scheduled debates between Trump and Hillary Clinton will be held on Sept. 26, with NBC’s Lester Holt as the journalist questioning the candidates.\n“I think it’s terrible,” Trump told Fox News Channel over the weekend. “They want the host to go after Trump.”\nHis statement is based on criticism NBC’s Matt Lauer received in some circles for being too easy on the Republican in a forum on national security earlier this month. Trump, who called Lauer “very professional,” told CNBC that he believes this puts pressure on other moderators to avoid Lauer’s fate by going after him.\nSports fans know the phenomenon as “working the refs.”\n“Trump’s buddy, the old basketball coach Bobby 

In [24]:
# Define a helper function to evaluate the baseline
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset['maintext']]
    return metric.compute(
        predictions=summaries, 
        references=dataset['title']
    )

In [25]:
# Evaluate the baseline on the validation set
import pandas as pd

score = evaluate_baseline(books_dataset['validation'], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': np.float64(13.23),
 'rouge2': np.float64(5.86),
 'rougeL': np.float64(11.23),
 'rougeLsum': np.float64(11.32)}

### Fine-tuning mT5 with the trainer API

In [26]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [40]:
# Directory settings
import os
os.environ["HF_HOME"] = "../data/cache"
os.environ["WANDB_DISABLED"] = "true"  # if not using wandb

# Traing arguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 16
num_train_epochs = 3
logging_steps = len(tokenized_books['train']) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"../data/models/{model_name}-finetuned-amazon-en-es",
    eval_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [49]:
# Set up metrics
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [57]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Handle tuple output
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    
    # Convert to numpy if tensor
    if hasattr(predictions, 'cpu'):
        predictions = predictions.cpu().numpy()
    if hasattr(labels, 'cpu'):
        labels = labels.cpu().numpy()
    
    # If 3D logits, get argmax
    if predictions.ndim == 3:
        predictions = np.argmax(predictions, axis=-1)
    
    # Replace invalid token IDs (-100 and out-of-vocab) with pad token
    vocab_size = tokenizer.vocab_size
    predictions = np.where(
        (predictions >= 0) & (predictions < vocab_size),
        predictions,
        tokenizer.pad_token_id
    )
    predictions = predictions.astype(np.int64)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = labels.astype(np.int64)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred)) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label)) for label in decoded_labels]
    
    result = rouge_score.compute(
        predictions=decoded_preds, 
        references=decoded_labels,
        use_stemmer=True,
    )
    
    return {k: round(v * 100, 4) for k, v in result.items()}

In [58]:
# Build up the collator
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer, 
    model=model,
)

In [43]:
tokenized_datasets = tokenized_datasets.remove_columns(
    books_dataset['train'].column_names
)


In [35]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 28241
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4701
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4699
    })
})

In [36]:
features = [tokenized_datasets['train'][i] for i in range(3)]
features

[{'input_ids': [18737,
   852,
   123810,
   274,
   10740,
   271,
   661,
   298,
   5219,
   5038,
   287,
   2262,
   421,
   54532,
   259,
   45007,
   261,
   20443,
   4396,
   339,
   421,
   37251,
   132341,
   263,
   351,
   259,
   21280,
   533,
   790,
   293,
   1578,
   390,
   259,
   55634,
   288,
   2354,
   955,
   287,
   276,
   1689,
   6320,
   259,
   55904,
   351,
   4065,
   260,
   563,
   259,
   262,
   12614,
   304,
   259,
   154144,
   910,
   287,
   9002,
   5219,
   261,
   287,
   259,
   85803,
   259,
   142165,
   265,
   1070,
   259,
   152986,
   345,
   533,
   359,
   1759,
   2974,
   339,
   259,
   5330,
   1418,
   31645,
   365,
   259,
   9825,
   4065,
   260,
   486,
   2262,
   304,
   7156,
   31499,
   285,
   33224,
   299,
   259,
   4964,
   4396,
   305,
   447,
   111266,
   56550,
   898,
   390,
   14461,
   351,
   53583,
   260,
   8156,
   514,
   259,
   85702,
   293,
   263,
   764,
   2828,
   102435,
   527,
  

In [59]:
# Instantiate the trainer
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_4370/2236513724.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [60]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.131900,3.148026,30.744200,15.675900,28.284700,28.251900
2,3.123700,2.673236,30.892800,15.778800,28.387900,28.338400
3,3.021100,2.666648,31.182100,15.939200,28.663700,28.626000


TrainOutput(global_step=5298, training_loss=2.7588231835377894, metrics={'train_runtime': 1689.2697, 'train_samples_per_second': 50.154, 'train_steps_per_second': 3.136, 'total_flos': 4.479714956000256e+16, 'train_loss': 2.7588231835377894, 'epoch': 3.0})

In [61]:
# Eval
trainer.evaluate()

{'eval_loss': 2.6666476726531982,
 'eval_rouge1': 31.1821,
 'eval_rouge2': 15.9392,
 'eval_rougeL': 28.6637,
 'eval_rougeLsum': 28.626,
 'eval_runtime': 94.1273,
 'eval_samples_per_second': 49.943,
 'eval_steps_per_second': 3.123,
 'epoch': 3.0}

In [62]:
# Push to the Hub
# Set up hf credentials
import os
from dotenv import load_dotenv
from huggingface_hub import HfApi, create_repo

load_dotenv()
token = os.getenv("HF_TOKEN_WRITE")

trainer.push_to_hub(tags="summarization", commit_message="First commit", token=token)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/tensor-polinomics/mt5-small-finetuned-amazon-en-es/commit/2081461e202a131146025890e79c937d2f9a5d35', commit_message='First commit', commit_description='', oid='2081461e202a131146025890e79c937d2f9a5d35', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tensor-polinomics/mt5-small-finetuned-amazon-en-es', endpoint='https://huggingface.co', repo_type='model', repo_id='tensor-polinomics/mt5-small-finetuned-amazon-en-es'), pr_revision=None, pr_num=None)

### Fine-tuning mT5 with Accelerate

In [63]:
# Set up data format to tensor
tokenized_datasets.set_format(
    type='torch', 
)

# Re-load the model
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint
)

# Define dataloaders
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    tokenized_datasets['train'], 
    shuffle=True, 
    batch_size=batch_size, 
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    tokenized_datasets['validation'], 
    batch_size=batch_size, 
    collate_fn=data_collator,
)

# Optimizer
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

# Accelerator
from accelerate import Accelerator
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# Learning rate scheduler
from transformers import get_scheduler
num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Postprocessing function
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
    return preds, labels

# Training loop
from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    
    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)
    
    result = rouge_score.compute()
    result = {key: round(value * 100, 4) for key, value in result.items()}
    print(f"Epoch {epoch}: {result}")



# Save to the Hub
model_name = "mt5-finetuned-en-es-accelerate"

# Create repo
repo_id = create_repo(model_name, token=token, exist_ok=True).repo_id
print(f"Repo created: {repo_id}")

accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(
    "../data/models/mt5-finetuned-en-es-accelerate",
    save_function=accelerator.save,
)
if accelerator.is_main_process:
    tokenizer.save_pretrained("../data/models/mt5-finetuned-en-es-accelerate")
    unwrapped_model.push_to_hub(model_name, token=token)
    tokenizer.push_to_hub(model_name, token=token)

  0%|          | 0/5298 [00:00<?, ?it/s]

/workspace/projects/hf_llm/.venv/lib/python3.12/site-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch 0: {'rouge1': np.float64(12.7479), 'rouge2': np.float64(3.8323), 'rougeL': np.float64(11.6365), 'rougeLsum': np.float64(11.6514)}
Epoch 1: {'rouge1': np.float64(14.8244), 'rouge2': np.float64(5.1632), 'rougeL': np.float64(13.5279), 'rougeLsum': np.float64(13.5317)}
Epoch 2: {'rouge1': np.float64(15.1101), 'rouge2': np.float64(5.3491), 'rougeL': np.float64(13.8103), 'rougeLsum': np.float64(13.8224)}
Repo created: tensor-polinomics/mt5-finetuned-en-es-accelerate


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            